# Nonlinear elasticity

The body of interest

In [ ]:
from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.webgui import Draw as DrawGeo
import ipywidgets as widgets

box = Box((0,0,0), (3,0.6,1))
box.faces.name="outer"
cyl = sum( [Cylinder((0.5+i,0,0.5), Y, 0.25,0.8) for i in range(3)] )
cyl.faces.name="cyl"
geo = box-cyl

ea = { "euler_angles" : [-70,5,30] }


cylboxedges = geo.faces["outer"].edges * geo.faces["cyl"].edges
cylboxedges.name = "cylbox"
geo = geo.MakeChamfer(cylboxedges, 0.03)

geo.faces.Min(X).name = "fix"
geo.faces.Max(X).name = "force"

Draw(geo, **ea)

mesh = Mesh(OCCGeometry(geo).GenerateMesh(maxh=0.1)).Curve(3)
scene = Draw (mesh, **ea)


In [ ]:
E, nu = 210, 0.2
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

force = CF( (1e-3,0,0) )


## Kinematic description and equilibrium of forces

Use the standard notation for nonlinear elasticity kinematics:
* material coordinates $X$, $Y$, $Z$
* displacement $\vec u$
* deformation gradient $\mathbf F = \mathbf I + \operatorname{Grad}\vec u$, $F_{ij} = \delta_{ij} + \partial u_i/\partial X_j$
* right Cauchy-Green tensor $\mathbf C = \mathbf F^T \cdot \mathbf F$
* Green strain $\mathbf E = \frac{1}{2} (\mathbf C - \mathbf I)$
* Jacobi determinant $J = \operatorname{det}\mathbf F$

Stresses and equilibrium:
* Cauchy stress tensor $\boldsymbol \sigma$ is symmetric, and in equilibrium with respect to spatial coordinates, $\operatorname{div}_x\boldsymbol \sigma = J \vec f$, $\boldsymbol \sigma \cdot \vec n = J_A \vec t$ on boundary
* first Piola-Kirchhoff stress tensor $\mathbf P$ is in equilibrium with respect to material coordinates, $\operatorname{div}_X\mathbf P= \vec f$, $\mathbf P \cdot \vec n = \vec t$ on boundary
* second Piola-Kirchhoff stress tensor $\mathbf S$ is symmetric
* relation $\mathbf P = \mathbf F \cdot \mathbf P$

Constitutive relations
* (hyper-)elastic energy density $\psi(\mathbf C)$ or $\psi(\mathbf F)$

$$\mathbf P = \frac{\partial \psi}{\partial \mathbf F}, \qquad \mathbf S = \frac{\partial \psi}{\partial \mathbf E} = 2\frac{\partial \psi}{\partial \mathbf C}$$

Principle of virtual work still holds

$$
\int_\Omega \delta \psi\, dV - \int_{\Gamma_N} \vec t\cdot \delta \vec u\, ds = 0.
$$

For St.Venant-Kirchhoff material with 

$$\psi = \mu \mathbf E : \mathbf E + \frac{\lambda}{2} (\operatorname{tr}\mathbf E)^2$$

implementation similar as before, but using Newton iteration.

In [ ]:
fes = VectorH1(mesh, order=3, dirichlet="fix")
u, deltau = fes.TnT()
gfu = GridFunction(fes)

def psi(F):
    E = 0.5*(F.trans*F - Id(mesh.dim))
    return mu*InnerProduct(E,E) + lam/2*Trace(E)**2


with TaskManager():
    a = BilinearForm(fes, symmetric=True)
    defgrad = Id(mesh.dim) + Grad(u)
    a += Variation(psi(defgrad)*dx)
    a += -force*deltau*ds("force")

    solvers.Newton(a, gfu, maxit=10)

For a small load, the solution is equivalent to the linear one. Visualize the stresses, computing them using the _Diff_ functionality.

In [ ]:
defgrad = Id(mesh.dim) + Grad(gfu)
P = psi(defgrad).Diff(defgrad)
scene = Draw(BoundaryFromVolumeCF(P[0]), mesh, deformation=gfu, **ea)

Now, apply a larger load in several load steps, using a compressible Neo-Hooke material law.

In [ ]:
import numpy as np

fes = VectorH1(mesh, order=2, dirichlet="fix")
u, deltau = fes.TnT()
gfu = GridFunction(fes)
loadpar = Parameter(0)

def psi(F):
    C = F.trans * F
    J = Det(F)
    return mu/2*(Trace(C)-3) - mu*log(J) + lam/2*log(J)**2

a = BilinearForm(fes, symmetric=True)
defgrad = Id(mesh.dim) + Grad(u)
a += Variation(psi(defgrad)*dx)
a += -loadpar*10000*force*deltau*ds("force")


gfu_history = GridFunction(fes, multidim=0)
loadsteps = np.linspace(0,1,11)
with TaskManager():
    for l in loadsteps:
        # print(f"load factor {l}")
        loadpar.Set(l)
        solvers.Newton(a, gfu, maxit=10, printing=False)

        gfu_history.AddMultiDimComponent(gfu.vec)

In [ ]:
scene_hist = Draw (gfu_history, mesh, animate=True, min=0, max=0.2, autoscale=True, deformation=True, **ea)

## Follower loads

cover page picture of:

[Nonlinear Solid Mechanics for Finite Element Analysis: Statics 1st Edition
by Javier Bonet (Author), Antonio J. Gil (Author), Richard D. Wood (Author)](https://www.amazon.com/Nonlinear-Mechanics-Finite-Element-Analysis/dp/1107115795/ref=pd_bxgy_thbs_d_sccl_1/144-7614425-7391713?pd_rd_w=CqCvS&content-id=amzn1.sym.c51e3ad7-b551-4b1a-b43c-3cf69addb649&pf_rd_p=c51e3ad7-b551-4b1a-b43c-3cf69addb649&pf_rd_r=9HSD8RKNRHDBSK8B5YKQ&pd_rd_wg=B0gaE&pd_rd_r=5fbf0386-d903-408e-b180-af0fd134e615&pd_rd_i=1107115795&psc=1)



In [ ]:
ea = { "euler_angles" : (-120,70,116) }

d = 0.01
box = Box ( (-d/2,-d/2,0), (d/2,d/2,0.1) ) + Box( (-d/2, -3*d/2,0.1), (d/2, 3*d/2, 0.1+d) )
box.faces.Min(Z).name = "bottom"
box.faces.Max(Z).name = "top"
Draw (box, **ea)

mesh = Mesh(OCCGeometry(box).GenerateMesh(maxh=0.005))
scene = Draw (mesh, **ea)

In [ ]:
E, nu = 210, 0.2
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

def F(u):
    return Id(u.dim) + Grad(u)

def C(u): 
    return F(u).trans*F(u)

def NeoHooke (C):
    return 0.5*mu*(Trace(C-Id(3)) + 2*mu/lam*Det(C)**(-lam/2/mu)-1)

The beam is twisted by a force applied to the top, the force rotating with the surface. Thus, it is a _follower load_, wich is *rotated* with the deformation gradient $\mathbf F$. As only the surface gradient is needed/available on the surface, make sure to use _Grad(u.Trace())_

In [ ]:
loadfactor = Parameter(1)
force = loadfactor * CF ( (-y, x, 0) )

fes = H1(mesh, order=3, dirichlet="bottom", dim=mesh.dim)
u,v = fes.TnT()

a = BilinearForm(fes, symmetric=True)
a += Variation(NeoHooke(C(u)).Compile()*dx)
a += (F(u.Trace())*force)*v*ds("top")

gfu = GridFunction(fes)
gfu.vec[:] = 0

In [ ]:
gfu_history = GridFunction(fes, multidim=0)
scene = Draw (gfu, deformation=True, **ea)
tw = widgets.Text(value='step = 0')
display(tw)

gfu.vec[:] = 0

numsteps = 30
for step in range(numsteps):
    loadfactor.Set(300*step/numsteps)
    solvers.Newton (a, gfu, printing=False, dampfactor=0.5)
    scene.Redraw()
    tw.value = 'step = '+str(step+1)+'/'+str(numsteps)
    gfu_history.AddMultiDimComponent(gfu.vec)

In [ ]:
scene_hist = Draw (gfu_history, mesh, animate=True, min=0, max=0.04, autoscale=True, deformation=True, **ea)